In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import nlppln
import ochre

with nlppln.WorkflowGenerator(working_dir='/home/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir=ochre.cwl_path())
    #print(wf.list_steps())
    
    txt_dir = wf.add_input(txt_dir='Directory')
    tei_dir = wf.add_input(tei_dir='Directory')
    repl = wf.add_input(replacement='string', default='space')
    tmp_ocr_dir_name = wf.add_input(tmp_ocr_dir_name='string', default='tmp_ocr')
    tmp_gs_dir_name = wf.add_input(tmp_gs_dir_name='string', default='tmp_gs')
    convert = wf.add_input(convert='boolean', default=True)
    
    tei_files = wf.ls(in_dir=tei_dir)
    unnormalized_gs_files = wf.tei2txt(tei_file=tei_files, scatter='tei_file', scatter_method='dotproduct')
    normalized_gs_files = wf.remove_newlines(in_file=unnormalized_gs_files, replacement=repl, 
                                             scatter='in_file', scatter_method='dotproduct')
    tmp_gs_dir = wf.save_files_to_dir(dir_name=tmp_gs_dir_name, in_files=normalized_gs_files)
    gs_without_empty = wf.delete_empty_files(in_dir=tmp_gs_dir)
    
    non_utf8_ocr_files = wf.ls(in_dir=txt_dir)
    unnormalized_ocr_files = wf.check_utf8(in_files=non_utf8_ocr_files, convert=convert)
    normalized_ocr_files = wf.remove_newlines(in_file=unnormalized_ocr_files, replacement=repl, 
                                              scatter='in_file', scatter_method='dotproduct')
    tmp_ocr_dir = wf.save_files_to_dir(dir_name=tmp_ocr_dir_name, in_files=normalized_ocr_files)
    ocr_without_empty = wf.delete_empty_files(in_dir=tmp_ocr_dir)
    
    tmp_gs_dir = wf.save_files_to_dir(dir_name=tmp_gs_dir_name, in_files=gs_without_empty)
    tmp_ocr_dir = wf.save_files_to_dir(dir_name=tmp_ocr_dir_name, in_files=ocr_without_empty)

    gs, ocr = wf.match_ocr_and_gs(gs_dir=tmp_gs_dir, ocr_dir=tmp_ocr_dir)

    wf.add_outputs(gs=gs)
    wf.add_outputs(ocr=ocr)
    
    wf.save(os.path.join(ochre.cwl_path(),'dbnl_ocr2ocr_and_gs.cwl'), mode='pack')